# PRIMER PARCIAL 02/2024

In [1]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import numpy as np
from scipy import optimize
import pandas as pd

In [19]:
# Cargar el dataset
data_path = '/content/drive/MyDrive/Colab Notebooks/IA/Primer Parcial/Cleaned_data_for_model.csv'
data = pd.read_csv(data_path)

# Normalizar datos numéricos para el entrenamiento.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[['price', 'baths', 'bedrooms', 'Area_in_Marla']] = scaler.fit_transform(data[['price', 'baths', 'bedrooms', 'Area_in_Marla']])

# Convertir columnas categóricas en variables dummy (variables indicadoras).
data = pd.get_dummies(data, columns=['property_type', 'location', 'city', 'purpose'], drop_first=True)

# Convertir a un array de NumPy.
data_array = data.values

In [20]:
from sklearn.model_selection import train_test_split

# Separar características (X) y etiquetas (y)
X = data_array[:, :-1]  # Todas las columnas menos la última.
y = data_array[:, -1]   # Última columna como etiqueta.

# Dividir el dataset en entrenamiento (80%) y prueba (20%).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Añadir la columna de unos para el término bias.
X_train = np.concatenate([np.ones((X_train.shape[0], 1)), X_train], axis=1)
X_test = np.concatenate([np.ones((X_test.shape[0], 1)), X_test], axis=1)

# Verificar las dimensiones.
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_test:", y_test.shape)


Dimensiones de X_train: (79599, 1404)
Dimensiones de y_train: (79599,)
Dimensiones de X_test: (19900, 1404)
Dimensiones de y_test: (19900,)


In [21]:
# Función sigmoide.
def sigmoid(z):
    """Función sigmoide."""
    return 1 / (1 + np.exp(-z))

# Función de coste con regularización.
def costFunction(theta, X, y, lambda_):
    """Función de coste para la regresión logística."""
    theta = np.array(theta, dtype=np.float64)
    X = np.array(X, dtype=np.float64)
    y = np.array(y, dtype=np.float64)

    m = y.size
    h = sigmoid(X.dot(theta))
    cost = (-1 / m) * (y.dot(np.log(h)) + (1 - y).dot(np.log(1 - h)))
    reg = (lambda_ / (2 * m)) * np.sum(np.square(theta[1:]))  # Regularización
    return cost + reg

# Función de gradiente con regularización.
def gradient(theta, X, y, lambda_):
    """Función de gradiente."""
    theta = np.array(theta, dtype=np.float64)
    X = np.array(X, dtype=np.float64)
    y = np.array(y, dtype=np.float64)

    m = y.size
    h = sigmoid(X.dot(theta))
    grad = (1 / m) * X.T.dot(h - y)
    reg = np.concatenate([[0], (lambda_ / m) * theta[1:]])  # Regularización para no sumar el bias.
    return grad + reg


In [22]:
# Definir la función para entrenar múltiples modelos de regresión logística (One-vs-All).
def oneVsAll(X, y, num_labels, lambda_):
    """Entrenar múltiples modelos de regresión logística (One-vs-All)."""
    X = np.array(X, dtype=np.float64)
    y = np.array(y, dtype=np.float64)

    m, n = X.shape
    all_theta = np.zeros((num_labels, n), dtype=np.float64)
    for i in range(num_labels):
        initial_theta = np.zeros(n, dtype=np.float64)
        # Entrenar usando optimización (minimización de coste).
        theta = optimize.fmin_tnc(func=costFunction, x0=initial_theta, fprime=gradient, args=(X, (y == i).astype(int), lambda_))[0]
        all_theta[i] = theta
    return all_theta

# Número de etiquetas (suponiendo que las etiquetas son enteros consecutivos).
num_labels = len(np.unique(y_train))  # Ajustar según el dataset.

# Parámetro de regularización.
lambda_ = 0.1

# Entrenar el modelo y obtener los parámetros para cada clase.
all_theta = oneVsAll(X_train, y_train, num_labels, lambda_)
print("Parámetros entrenados para cada clase (all_theta):\n", all_theta)


<ipython-input-21-217f1f3d45e8>:15: RuntimeWarning: divide by zero encountered in log
  cost = (-1 / m) * (y.dot(np.log(h)) + (1 - y).dot(np.log(1 - h)))


Parámetros entrenados para cada clase (all_theta):
 [[-5.58267565e-06 -7.54182396e-06 -2.14390630e-05 ... -2.58229619e-06
  -7.27632528e-06  2.42951566e-07]
 [ 3.26541070e+00 -3.93199566e-06  9.22940474e+00 ...  8.21687513e-01
   2.23893359e+00  1.22332122e-02]]


In [23]:
# Función de predicción utilizando los parámetros entrenados (One-vs-All).
def predictOneVsAll(all_theta, X):
    """Predecir utilizando los parámetros entrenados."""
    X = np.array(X, dtype=np.float64)
    return np.argmax(sigmoid(X.dot(all_theta.T)), axis=1)

# Calcular predicciones en el conjunto de prueba.
predictions = predictOneVsAll(all_theta, X_test)

# Calcular la precisión del modelo.
accuracy = np.mean(predictions == y_test) * 100
print(f'Precisión del modelo en el conjunto de prueba: {accuracy:.2f}%')


Precisión del modelo en el conjunto de prueba: 99.42%
